In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def pad_and_concat(sequences):  # sequences shape: [batch_size, len, dims...] -> ([batch_size, maxlen, dims...], [len])
    arrays = [np.asarray(seq) for seq in sequences]
    lengths = np.asarray([array.shape[0] for array in arrays], dtype=np.int32)
    maxlen = np.max(lengths)
    arrays = [np.pad(array, [(0, maxlen - array.shape[0]), (0, 0)], 'constant', constant_values=0) for array in arrays]
    return np.asarray(arrays), lengths
    

In [3]:
small_cnf, small_lengths = pad_and_concat(
    [[[1, -2], [2, 1]],
     [[-2, -1], [1, -2]],
     [[-1, -1], [-2, -2], [-1, -2]],
      [[1, 1], [-1, -1]],
      [[1, -1], [1, 2], [-1, -2]],
      [[-2, -2], [2, 2]]
     ])
small_sol = np.asarray([1, 1, 1, 0, 1, 0])

In [4]:
print(small_cnf)
print(small_lengths)

[[[ 1 -2]
  [ 2  1]
  [ 0  0]]

 [[-2 -1]
  [ 1 -2]
  [ 0  0]]

 [[-1 -1]
  [-2 -2]
  [-1 -2]]

 [[ 1  1]
  [-1 -1]
  [ 0  0]]

 [[ 1 -1]
  [ 1  2]
  [-1 -2]]

 [[-2 -2]
  [ 2  2]
  [ 0  0]]]
[2 2 3 2 3 2]


In [5]:
VARIABLE_NUM = 3
EMBEDDING_SIZE = 8
CLAUSE_SIZE = 2
LSTM_STATE_SIZE = 8
batch_size = 6

In [6]:
def assert_shape(matrix, shape: list):
    act_shape = matrix.get_shape().as_list()
    assert act_shape == shape, "got shape {}, expected {}".format(act_shape, shape)

In [7]:
class Graph:
    def __init__(self):
        self.inputs = tf.placeholder(tf.int32, shape=(batch_size, None, CLAUSE_SIZE), name='inputs')
        self.lengths = tf.placeholder(tf.int32, shape=(batch_size,), name='lengths')
        self.labels = tf.placeholder(tf.float32, shape=(batch_size,), name='labels')
        
        vars_ = tf.abs(self.inputs)
        signs = tf.cast(tf.sign(self.inputs), tf.float32)  # shape: [batch_size, None, CLAUSE_SIZE]

        embeddings = tf.Variable(tf.random_uniform([VARIABLE_NUM + 1, EMBEDDING_SIZE], -1., 1), name='embeddings')

        var_embeddings = tf.nn.embedding_lookup(embeddings, vars_)
        # var_embeddings shape: [None, None, CLAUSE_SIZE, EMBEDDING_SIZE]
        
        clause_preembeddings = tf.concat(
            [tf.reshape(var_embeddings, [batch_size, -1, CLAUSE_SIZE * EMBEDDING_SIZE]), 
             signs],
            axis=2)
        
        PREEMBEDDING_SIZE = EMBEDDING_SIZE * CLAUSE_SIZE + CLAUSE_SIZE
        assert_shape(clause_preembeddings, 
                     [batch_size, None, PREEMBEDDING_SIZE])
        
        clause_w = tf.Variable(tf.random_normal(
            [PREEMBEDDING_SIZE, EMBEDDING_SIZE]), name='clause_w')
        clause_b = tf.Variable(tf.random_normal([EMBEDDING_SIZE]), name='clause_b')
        clause_embeddings = tf.reshape(tf.sigmoid(
            tf.reshape(clause_preembeddings, [-1, PREEMBEDDING_SIZE]) @ clause_w + clause_b), 
                                       [batch_size, -1, EMBEDDING_SIZE])
        # shape: [None, None, EMBEDDING_SIZE]
        
        lstm = tf.contrib.rnn.BasicLSTMCell(LSTM_STATE_SIZE)
        hidden_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        current_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        state = hidden_state, current_state
        
        _, lstm_final_state = tf.nn.dynamic_rnn(lstm, clause_embeddings, dtype=tf.float32, 
                                               sequence_length=self.lengths
                                               )
        formula_embedding = lstm_final_state.h
            
        assert_shape(formula_embedding, [batch_size, LSTM_STATE_SIZE])
            
        softmax_w = tf.Variable(tf.random_normal([LSTM_STATE_SIZE, 1]), name='softmax_w')
        softmax_b = tf.Variable(tf.random_normal([1]), name='softmax_b')
        
        self.logits = tf.squeeze(formula_embedding @ softmax_w, axis=1) + softmax_b
        self.loss = tf.losses.sigmoid_cross_entropy(self.labels, self.logits) 
        self.probabilities = tf.sigmoid(self.logits)

tf.reset_default_graph()
model = Graph()

In [8]:
np.set_printoptions(precision=2, suppress=True)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    probs = sess.run([model.probabilities], feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
print(probs)

[array([0.87, 0.88, 0.87, 0.74, 0.82, 0.92], dtype=float32)]


In [10]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        sess.run(train_op, feed_dict={
            model.inputs: small_cnf,
            model.labels: small_sol,
            model.lengths: small_lengths
        })
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
probs

array([1.  , 1.  , 1.  , 0.01, 0.01, 0.  ], dtype=float32)

In [11]:
import cnf

In [12]:
def sat_array():
    for k in range(1, 6):
        for var_num in range(2, 5):
            for clause_num in range(3, 10):
                sat = {True: 0, False: 0}
                for _ in range(100):
                    sat[cnf.get_random_kcnf(k, var_num, clause_num).satisfiable()] += 1
                print(k, var_num, clause_num, sat)
#sat_array()

In [19]:
from sklearn.utils import shuffle
def generate_balanced_set():
    sat = {True: [], False: []}
    for formula in cnf.get_random_kcnfs(30000, 2, 3, 7):
        sat[formula.satisfiable()].append(formula)
        
    minlen = min(len(sat[True]), len(sat[False]))
    del sat[True][minlen:]
    del sat[False][minlen:]
    cnfs = sat[True] + sat[False]
    labels = [True] * minlen + [False] * minlen
    shuffle(cnfs, labels)
    return cnfs, labels

In [20]:
from sklearn.model_selection import train_test_split
train_cnfs, test_cnfs, train_labels, test_labels = train_test_split(*generate_balanced_set(), test_size=0.1)

In [21]:
print("train sat:", sum(1 for label in train_labels if label == True), "train unsat:", sum(1 for label in train_labels if label == False))
print("test sat:", sum(1 for label in test_labels if label == True), "train unsat:", sum(1 for label in test_labels if label == False))

train sat: 3209 train unsat: 3191
test sat: 347 train unsat: 365


In [22]:
def chunks(lists, chunk_size):
    return [[it[i:i + chunk_size] for it in lists] for i in range(0, len(lists[0]), chunk_size)]

In [23]:
chunks([list(range(10)), list(range(10, 20))], 2)

[[[0, 1], [10, 11]],
 [[2, 3], [12, 13]],
 [[4, 5], [14, 15]],
 [[6, 7], [16, 17]],
 [[8, 9], [18, 19]]]

In [25]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    
    for epoch_num in range(1000):
        print("Epoch", epoch_num)
        losses = []
        accs = []
        for (batch_cnfs, batch_labels) in chunks((train_cnfs, train_labels), batch_size):
            if len(batch_cnfs) < batch_size:
                print("skipping incomplete batch")
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            _, loss, probs = sess.run([train_op, model.loss, model.probabilities], feed_dict={
                model.inputs: inputs,
                model.labels: batch_labels,
                model.lengths: lengths
            })
            losses.append(loss)
            accs.append(1 - np.mean(np.abs(np.around(probs) - np.asarray(batch_labels))))
            
        print("train loss:", np.mean(np.asarray(losses)), "train acc:", np.mean(accs))
        
        losses = []
        accs = []
        for (batch_cnfs, batch_labels) in chunks((test_cnfs, test_labels), batch_size):
            if len(batch_cnfs) < batch_size:
                print("skipping incomplete batch")
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            loss, probs = sess.run([model.loss, model.probabilities], feed_dict={
                model.inputs: inputs,
                model.lengths: lengths,
                model.labels: batch_labels
            })
            losses.append(loss)
            accs.append(1 - np.mean(np.abs(np.around(probs) - np.asarray(batch_labels))))
        print('test loss:', np.mean(np.asarray(losses)), 'test acc:', np.mean(accs))
        
        print("first train batch:")
        first_inputs, first_lengths = pad_and_concat([cnf.clauses for cnf in train_cnfs[:batch_size]])
        probs = sess.run(model.probabilities, feed_dict={
            model.inputs: first_inputs,
            model.lengths: first_lengths
        })
        print("actual:", probs)
        print("expected", train_labels[:batch_size])
        
        print("first test batch:")
        first_inputs, first_lengths = pad_and_concat([cnf.clauses for cnf in test_cnfs[:batch_size]])
        probs = sess.run(model.probabilities, feed_dict={
            model.inputs: first_inputs,
            model.lengths: first_lengths
        })
        print("actual:", probs)
        print("expected", test_labels[:batch_size])

        print("small dataset:")
        probs = sess.run(model.probabilities, feed_dict={
            model.inputs: small_cnf,
            model.lengths: small_lengths
        })
        print("actual", probs)
        print("expected", small_sol)




Epoch 0
skipping incomplete batch
train loss: 0.4998477 train acc: 0.7546904263755841
skipping incomplete batch
test loss: 0.42768028 test acc: 0.8192090354228424
first train batch:
actual: [0.25 0.33 0.24 0.24 0.54 0.39]
expected [False, False, False, False, False, True]
first test batch:
actual: [0.32 0.9  0.69 0.28 0.52 0.22]
expected [False, True, True, False, True, False]
small dataset:
actual [0.98 0.97 0.53 0.79 0.9  0.76]
expected [1 1 1 0 0 0]
Epoch 1
skipping incomplete batch
train loss: 0.417555 train acc: 0.8047216963617037
skipping incomplete batch
test loss: 0.38770297 test acc: 0.8305084705352783
first train batch:
actual: [0.17 0.26 0.16 0.18 0.48 0.24]
expected [False, False, False, False, False, True]
first test batch:
actual: [0.19 0.98 0.69 0.23 0.5  0.16]
expected [False, True, True, False, True, False]
small dataset:
actual [0.99 0.99 0.41 0.69 0.98 0.72]
expected [1 1 1 0 0 0]
Epoch 2
skipping incomplete batch
train loss: 0.40125838 train acc: 0.8133208210987177


KeyboardInterrupt: 